# Velocity autocorrelation
[ref](https://en.wikipedia.org/wiki/Autocovariance)

note: yupi vacf in time_avg mode performs autocovariance actually, not the autocorrelation

$$ K_{XX}(t_1, t_2) = cov \big[X_{t_1} , X_{t_2} \big] = E \big[ (X_{t_1} - \mu_{t_1}) (X_{t_2} - \mu_{t_2}) \big] = E \big[ X_{t_1}, X_{t_2} \big] - \mu_{t_1} \mu_{t_2} $$


$$ K_{XX}(\tau) = E \big[ (X_t - \mu_t) (X_{t-\tau} - \mu_{t-\tau}) \big] $$

modified yupi source code at ./pyenv/lib/python3.10/site-packages/yupi/stats/_stats.py, 
original :

    for lag_ in range(1, lag + 1):
        # Multiply components given lag
        v1, v2 = v[:-lag_], v[lag_:]
        v1v2 = (v1 - v1.mean(axis=0)) * (v2 - v2.mean(axis=0))

        # Dot product for a given lag time
        v1_dot_v2 = np.sum(v1v2, axis=1)

        # Averaging over a single realization
        current_vacf[lag_ - 1] = np.mean(v1_dot_v2)
        
modified:

    for lag_ in range(lag):
        if lag_ == 0: v1, v2 = v[:], v[:]
        else: v1, v2 = v[:-lag_], v[lag_:]
        # Multiply components given lag
        v1v2 = (v1 - v1.mean(axis=0)) * (v2 - v2.mean(axis=0))

        # Dot product for a given lag time
        v1_dot_v2 = np.sum(v1v2, axis=1)

        # Averaging over a single realization
        current_vacf[lag_] = np.mean(v1_dot_v2)

In [ ]:
maxLagtime = 1000
x = np.arange(0, 100, 0.1) 

blueTrajs, redTraj = get_trajs(nDrops, red_particle_idx, rawTrajs)
blueTrajs_smooth, redTraj_smooth = get_trajs(nDrops, red_particle_idx, smoothTrajs)

In [ ]:
print("Global Velocity Autocovariance Function")
vacf_b, vacf_std_b = ys.vacf(blueTrajs, time_avg=True, lag = maxLagtime)
vacf_r, vacf_std_r = ys.vacf(redTraj, time_avg=True, lag = maxLagtime)

vacf_b_smooth, vacf_std_b_smooth = ys.vacf(blueTrajs_smooth, time_avg=True, lag = maxLagtime)
vacf_r_smooth, vacf_std_r_smooth = ys.vacf(redTraj_smooth, time_avg=True, lag = maxLagtime)

In [ ]:
fig, (ax, ax1) = plt.subplots(2, 1, figsize=(10, 5))
ax.errorbar(x, vacf_b, fmt='o', markersize = 1, color = "blue", label = 'blue particles')
ax.fill_between(x, vacf_b + vacf_std_b, vacf_b - vacf_std_b, alpha=1, edgecolor='#F0FFFF', facecolor='#00FFFF')
ax.grid()
ax.legend()
ax.set(xlim = (-1, 10), xlabel = 'Lag time [s]', ylabel = r'VACF [$(px/s)^2$]')
ax1.errorbar(x, vacf_r, yerr = vacf_std_r, fmt='o',
                markersize=1, color = "red", label = 'red particle')
ax1.set(xlim = (-1, 10), xlabel = 'Lag time [s]', ylabel = r'VACF [$(px/s)^2$]')
ax1.grid()
ax1.legend()
plt.suptitle("Velocity autocorrelation function - Raw trajectories")
plt.tight_layout()
if save_verb: plt.savefig(f"./{res_path}/velocity_autocovariance/vacf_raw.png", )
if show_verb: 
    plt.show()
else:
    plt.close()

fig, (ax, ax1) = plt.subplots(2, 1, figsize=(10, 5))
ax.errorbar(x, vacf_b_smooth, fmt='o', markersize = 1, color = "blue", label = 'blue particles')
ax.fill_between(x, vacf_b_smooth + vacf_std_b_smooth, vacf_b_smooth - vacf_std_b_smooth,
                    alpha=1, edgecolor='#F0FFFF', facecolor='#00FFFF')
ax.set(xlim=(-1, 10), xlabel = 'Lag time [s]', ylabel = r'VACF [$(px/s)^2$]')
ax.grid()
ax.legend()
ax1.errorbar(x, vacf_r_smooth, yerr=vacf_std_r_smooth, fmt='o',
                 markersize=1, color = "red", label = 'red particle')
ax1.set(xlim=(-1, 10), xlabel = 'Lag time [s]', ylabel = r'VACF [$(px/s)^2$]')
ax1.grid()  
ax1.legend()
plt.suptitle("Velocity autocorrelation function - Smooth trajectories")
plt.tight_layout()
if save_verb: plt.savefig(f"./{res_path}/velocity_autocovariance/vacf_smooth.png", )
if show_verb: 
    plt.show()
else:
    plt.close()

## Windowed Analysis 

In [ ]:
def vacf_vindowed(overwrite, trajectories, raw):        
    vacf_b_wind = []
    vacf_b_std_wind = []
    vacf_r_wind = []
    vacf_sigmas = np.zeros((nSteps, 2))

    for k in tqdm(range(nSteps)):
        trajs = trajectories.loc[trajectories.frame.between(startFrames[k], endFrames[k])]
        blueTrajs, redTraj = get_trajs(nDrops, red_particle_idx, trajs)

        temp = ys.vacf(blueTrajs, time_avg = True, lag = maxLagtime)
        vacf_b_wind.append(temp[0])
        vacf_b_std_wind.append(temp[1])

        temp  = ys.vacf(redTraj, time_avg = True, lag = maxLagtime)
        vacf_r_wind.append(temp[0])

    vacf_b_wind = pd.DataFrame(vacf_b_wind)
    vacf_b_std_wind = pd.DataFrame(vacf_b_std_wind)
    vacf_r_wind = pd.DataFrame(vacf_r_wind)
    
    vacf_r_wind.columns = vacf_r_wind.columns.astype(str)
    vacf_b_wind.columns = vacf_b_wind.columns.astype(str)
    vacf_b_std_wind.columns = vacf_b_std_wind.columns.astype(str)

    if overwrite:
        if raw: 
            path = f"./{analysis_data_path}/vacf/raw/"
        else: 
            path = f"./{analysis_data_path}/vacf/smooth/"
    
        vacf_b_wind.to_csv(path + "vacf_blue_wind.csv")
        vacf_b_std_wind.to_csv(path + "vacf_blue_std_wind.csv")
        vacf_r_wind.to_csv(path + "vacf_red_wind.csv")
        
    return vacf_b_wind, vacf_b_std_wind, vacf_r_wind

In [ ]:
print("Windowed analysis")
if run_windowed_analysis:
    vacf_b_wind, vacf_b_std_wind, vacf_r_wind = vacf_vindowed(overwrite, rawTrajs, True)
    vacf_b_wind_smooth, vacf_b_std_wind_smooth, vacf_r_wind_smooth = vacf_vindowed(overwrite, smoothTrajs, False)

else:
    path = f"./{analysis_data_path}/vacf/raw/"
    vacf_b_wind = pd.read_csv(path + "vacf_blue_wind.csv", index_col=0)
    vacf_b_std_wind = pd.read_csv(path + "vacf_blue_std_wind.csv", index_col=0)
    vacf_r_wind = pd.read_csv(path + "vacf_red_wind.csv", index_col=0)

    path = f"./{analysis_data_path}/vacf/smooth/"
    vacf_b_wind_smooth = pd.read_csv(path + "vacf_blue_wind.csv", index_col=0)
    vacf_b_std_wind_smooth = pd.read_csv(path + "vacf_blue_std_wind.csv", index_col=0)
    vacf_r_wind_smooth = pd.read_csv(path + "vacf_red_wind.csv", index_col=0)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(startFrames/10, vacf_b_wind["0"], 'b', label = "Blue particles")
ax.plot(startFrames/10, vacf_r_wind["0"], 'r', label = "Red particle")
ax.set(title = "Time evolution of velocity variance - Raw Trajectories", ylabel = "$\sigma$", xlabel = "Window Time [s]")
ax.grid()
ax.legend()
if save_verb: plt.savefig(f"./{res_path}/velocity_autocovariance/vacf_wind_0_raw.png", )
if show_verb:
    plt.show()
else: 
    plt.close()

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(startFrames/10, vacf_b_wind_smooth["0"], 'b', label = "Blue particles")
ax.plot(startFrames/10, vacf_r_wind_smooth["0"], 'r', label = "Red particle")
ax.set(title = "Time evolution of velocity variance - Smooth Trajectories ", ylabel = "$\sigma$", xlabel = "Window Time [s]")
ax.grid()
ax.legend()
if save_verb: plt.savefig(f"./{res_path}/velocity_autocovariance/vacf_wind_0_smooth.png", )
if show_verb:
    plt.show()
else: 
    plt.close()

In [ ]:
if animated_plot_verb:
    fig = plt.figure(figsize=(8, 5))
    anim_running = True

    def onClick(event):
        global anim_running
        if anim_running:
            ani.event_source.stop()
            anim_running = False
        else:
            ani.event_source.start()
            anim_running = True

    def update_graph(step):
        line.set_ydata(vacf_b_wind.iloc[step])
        title.set_text(f"Velocity autocorrelation - Raw Trajectories - window [{startFrames[step]/10} - {endFrames[step]/10}] s")
        line1.set_ydata(vacf_r_wind.iloc[step])
        return line, line1,

    ax = fig.add_subplot(211)
    title = ax.set_title(f"Velocity autocorrelation - Raw Trajectories - window [{startFrames[0]/10} - {endFrames[0]/10}] s")
    line, = ax.plot(np.arange(0.1, 100.1, 0.1), vacf_b_wind.iloc[0], 'b-', label = 'Blue particles')
    ax.set(ylabel = r'vacf [$(px/s)^2$]', xlabel = 'lag time $t$ [s]', xlim = (-1, 20))
    ax.grid()
    ax.legend()

    ax1 = fig.add_subplot(212)

    line1, = ax1.plot(np.arange(0.1, 100.1, 0.1), vacf_r_wind.iloc[0], 'r-', label = 'Red particle')
    ax1.set(ylabel = r'vacf [$(px/s)^2$]', xlabel = 'lag time $t$ [s]', xlim = (-1, 20))
    ax1.grid()
    ax1.legend()

    plt.tight_layout()
    fig.canvas.mpl_connect('button_press_event', onClick)
    ani = matplotlib.animation.FuncAnimation(fig, update_graph, nSteps, interval = 5, blit=False)
    if save_verb: ani.save(f'./{res_path}/velocity_autocovariance/vacf_wind_raw.mp4', fps=30, extra_args=['-vcodec', 'libx264'])
    if anim_show_verb:
        plt.show()
    else:
        plt.close()

    fig = plt.figure(figsize=(8, 5))
    anim_running = True

    def onClick(event):
        global anim_running
        if anim_running:
            ani.event_source.stop()
            anim_running = False
        else:
            ani.event_source.start()
            anim_running = True

    def update_graph(step):
        line.set_ydata(vacf_b_wind_smooth.iloc[step])
        title.set_text(f"Velocity autocorrelation - Smooth Trajectories - window [{startFrames[step]/10} - {endFrames[step]/10}] s")
        line1.set_ydata(vacf_r_wind_smooth.iloc[step])
        return line, line1,

    ax = fig.add_subplot(211)
    title = ax.set_title(f"Velocity autocorrelation - Smooth Trajectories - window [{startFrames[0]/10} - {endFrames[0]/10}] s")
    line, = ax.plot(np.arange(0.1, 100.1, 0.1), vacf_b_wind_smooth.iloc[0], 'b-', label = 'Blue particles')
    ax.set(ylabel = r'vacf [$(px/s)^2$]', xlabel = 'lag time $t$ [s]', xlim = (-1, 20))
    ax.grid()
    ax.legend()

    ax1 = fig.add_subplot(212)
    line1, = ax1.plot(np.arange(0.1, 100.1, 0.1), vacf_r_wind_smooth.iloc[0], 'r-', label = 'Red particle')
    ax1.set(ylabel = r'vacf [$(px/s)^2$]', xlabel = 'lag time $t$ [s]', xlim = (-1, 20))
    ax1.grid()
    ax1.legend()

    plt.tight_layout()
    fig.canvas.mpl_connect('button_press_event', onClick)
    ani = matplotlib.animation.FuncAnimation(fig, update_graph, nSteps, interval = 5, blit=False)
    if save_verb: ani.save(f'./{res_path}/velocity_autocovariance/vacf_wind_smooth.mp4', fps=30, extra_args=['-vcodec', 'libx264'])
    if anim_show_verb:
        plt.show()
    else:
        plt.close()